In [1]:
import pandas as pd
import json
import requests
import glob
import os
from sqlalchemy import create_engine
from bs4 import BeautifulSoup
import time

## Get the Data

### -> Data in files

In [2]:
!ls data/data_0.tsv

data/data_0.tsv


In [3]:
def show_dtypes_na(df: pd.DataFrame) -> pd.DataFrame: 
    df_show = pd.concat(
        [
            df.dtypes.sort_index(), 
            df.isna().sum().sort_index()
        ],
        axis=1
    ).rename({0: 'dtypes', 1: 'na'}, axis=1)
    
    return df_show

#### - slice-0: tsv file, compressed 

In [4]:
slice_0 = pd.read_csv('data/data_0.tsv', sep='\t', compression='gzip')
#slice_0.columns = slice_0.columns.str.lower()
slice_0 = slice_0.drop('Unnamed: 0', axis=1)
slice_0 = slice_0.dropna(subset=['Timestamp'])
# id is an int
slice_0.Timestamp = slice_0.Timestamp.astype(int)
slice_0

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
124994,1332817560,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124995,1332817620,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124996,1332817680,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124997,1332817740,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
show_dtypes_na(slice_0)

,dtypes,na
Close,float64,123463
High,float64,123463
Low,float64,123463
Open,float64,123463
Timestamp,int64,0
Volume_(BTC),float64,123463
Volume_(Currency),float64,123463
Weighted_Price,float64,123463


#### - slice-1: csv with Nones and repeated index

In [6]:
pd.set_option('display.max_columns', 500)

slice_1 = pd.read_csv('data/data_1.csv', sep=',')
# drop rows with no id
slice_1 = slice_1.dropna(subset=['Timestamp'])


slice_1["Timestamp"] = pd.to_datetime(slice_1["Timestamp"]).apply(lambda x: int(time.mktime(x.timetuple())))
# id is an int
slice_1.Timestamp = slice_1.Timestamp.astype(int)
# drop row numbers column
slice_1 = slice_1.drop('Unnamed: 0', axis=1)
slice_1.head(10)

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1332814260,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1332814320,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1332814380,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1332814440,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1332814500,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1332814560,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1332814620,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1332814680,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1332814740,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1332814800,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
show_dtypes_na(slice_1)

,dtypes,na
Close,float64,121895
High,float64,121895
Low,float64,121895
Open,float64,121895
Timestamp,int64,0
Volume_(BTC),float64,121895
Volume_(Currency),float64,121895
Weighted_Price,float64,121895


#### - slice-2: JSON file with Nones, values converted to strings

In [8]:
pd.set_option('display.max_columns', 500)

slice_2 = pd.read_csv('data/data_2.csv', sep=',')
# drop rows with no id
slice_2 = slice_2.dropna(subset=['timestamp'])
# id is an int
slice_2.timestamp = slice_2.timestamp.astype(int)
# drop row numbers column
slice_2 = slice_2.drop('Unnamed: 0', axis=1)
slice_2.head(10)

,timestamp,open,high,low,close,volume_(btc),volume_(currency),weighted_price,volumn_square
0,1340317800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1340317860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1340317920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1340317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1340318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1340318100,6.64,6.64,6.64,6.64,2.019841,13.411742,6.64,4.079757
6,1340318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1340318220,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1340318280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1340318340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### slice-3: JSON file with Nones, values converted to strings, multiple lines

In [9]:

slice_3 = pd.read_json('data/data_3.json' ,compression='bz2')
#slice_3 = file_path = os.path.join('data', 'data_3.json')
slice_3['Timestamp'] = slice_3.Timestamp.apply(lambda x: int(time.mktime(x.timetuple())))
slice_3

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1347814140,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1347814200,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1347814260,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1347814320,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1347814380,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
124994,1355317380,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124995,1355317440,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124996,1355317500,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124997,1355317560,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
show_dtypes_na(slice_3)

,dtypes,na
Close,float64,113964
High,float64,113964
Low,float64,113964
Open,float64,113964
Timestamp,int64,0
Volume_(BTC),float64,113964
Volume_(Currency),float64,113964
Weighted_Price,float64,113964


#### Fix issues on particular dataframes

In [11]:
# change all column names to lower 
slice_0.columns= slice_0.columns.str.lower()
slice_1.columns= slice_1.columns.str.lower()
slice_2.columns= slice_2.columns.str.lower()
slice_3.columns= slice_3.columns.str.lower()


In [12]:
# set the index
slice_0 = slice_0.set_index('timestamp') 
slice_1 = slice_1.set_index('timestamp') 
slice_2 = slice_2.set_index('timestamp') 
slice_3 = slice_3.set_index('timestamp') 

slice_0 = slice_0.dropna(how ='all')
slice_1 = slice_1.dropna(how ='all')
slice_2 = slice_2.dropna(how ='all')
slice_3 = slice_3.dropna(how ='all')


In [13]:
# concatenate dataframes - by default, pd.concat uses an outer join on the index. pd.merge could be an alternative
file_data_df = pd.concat([slice_0, slice_1, slice_2, slice_3]).sort_index()
print("Concatenated shape: ", file_data_df.shape)

Concatenated shape:  (24521, 8)


In [14]:
# for the same id
# it looks like the values either agree with each other or are null
file_data_df.sort_index()

,open,high,low,close,volume_(btc),volume_(currency),weighted_price,volumn_square
timestamp,,,,,,,,
1325317920,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000,NaN
1325346600,4.39,4.39,4.39,4.39,48.000000,210.720000,4.390000,NaN
1325350740,4.50,4.57,4.50,4.57,37.862297,171.380337,4.526411,NaN
1325350800,4.58,4.58,4.58,4.58,9.000000,41.220000,4.580000,NaN
1325391360,4.58,4.58,4.58,4.58,1.502000,6.879160,4.580000,NaN
...,...,...,...,...,...,...,...,...
1355314260,13.34,13.34,13.34,13.34,24.866857,331.723868,13.340000,NaN
1355314320,13.33,13.33,13.33,13.33,16.881552,225.031092,13.330000,NaN
1355315400,13.42,13.42,13.42,13.42,5.763040,77.340000,13.420000,NaN


In [15]:
# so lets fill the null values (within the same column and the same id) 
# with any non null value for the same id
#file_data_df = file_data_df.fillna(method='ffill').fillna(method='bfill').sort_index()
#file_data_df

In [16]:
# then keep only 1 row per id
file_data_df = file_data_df.groupby(file_data_df.index).first()
file_data_df

,open,high,low,close,volume_(btc),volume_(currency),weighted_price,volumn_square
timestamp,,,,,,,,
1325317920,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000,NaN
1325346600,4.39,4.39,4.39,4.39,48.000000,210.720000,4.390000,NaN
1325350740,4.50,4.57,4.50,4.57,37.862297,171.380337,4.526411,NaN
1325350800,4.58,4.58,4.58,4.58,9.000000,41.220000,4.580000,NaN
1325391360,4.58,4.58,4.58,4.58,1.502000,6.879160,4.580000,NaN
...,...,...,...,...,...,...,...,...
1355314260,13.34,13.34,13.34,13.34,24.866857,331.723868,13.340000,NaN
1355314320,13.33,13.33,13.33,13.33,16.881552,225.031092,13.330000,NaN
1355315400,13.42,13.42,13.42,13.42,5.763040,77.340000,13.420000,NaN


In [17]:
# check missing data 
file_data_df.isnull().sum()

open                     0
high                     0
low                      0
close                    0
volume_(btc)             0
volume_(currency)        0
weighted_price           0
volumn_square        15675
dtype: int64

In [18]:
#file_data_df.dropna()

In [19]:
file_data_df_final=file_data_df.copy()

In [20]:
file_data_df.to_csv('file_data_df_final')

/home/francisco/.virtualenvs/slu01/lib/python3.8/site-packages/pandas/core/internals/blocks.py:2199: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


In [21]:
file_data_df_final.sort_index()

,open,high,low,close,volume_(btc),volume_(currency),weighted_price,volumn_square
timestamp,,,,,,,,
1325317920,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000,NaN
1325346600,4.39,4.39,4.39,4.39,48.000000,210.720000,4.390000,NaN
1325350740,4.50,4.57,4.50,4.57,37.862297,171.380337,4.526411,NaN
1325350800,4.58,4.58,4.58,4.58,9.000000,41.220000,4.580000,NaN
1325391360,4.58,4.58,4.58,4.58,1.502000,6.879160,4.580000,NaN
...,...,...,...,...,...,...,...,...
1355314260,13.34,13.34,13.34,13.34,24.866857,331.723868,13.340000,NaN
1355314320,13.33,13.33,13.33,13.33,16.881552,225.031092,13.330000,NaN
1355315400,13.42,13.42,13.42,13.42,5.763040,77.340000,13.420000,NaN
